In [32]:
import pandas as pd
import numpy as np
from csv import reader
from graphviz import Digraph
import pickle

In [33]:
# path to the KG in csv format
raw = "./raw/cleaned_humans_8k.csv"
# path to the mapping file
mapping = "./mapping/cleaned_humans.csv"

In [34]:
class Node:
    __counter = 0
    
    # all properties of the loaded dataframe of a node
    def __properties(self):
        return self.__df['p']
    
    # all subjects of the loaded dataframe of a node
    def __subjects(self):
        return self.__df['s']
    
    # all distinct properties of the loaded dataframe of a node
    def __properties_unique(self):
        return self.__properties().unique()
    
    # all distinct subjects of the loaded dataframe of a node
    def __subjects_unique(self):
        return self.__subjects().unique()
    
    # the *n* most used properties of the loaded dataframe of a node
    def __most_used_properties(self, n=10):
        return self.__properties().value_counts()[0:n].index
        
    # the rows that use the most used property of the loaded dataframe of a node
    def __rows_for_most_used_property(self):
        return self.__df.loc[self.__df['p'] == self.__most_used_property]
 
    # the rows of subjects that use the most used property of the loaded dataframe of a node
    def __rows_for_subjects_using_most_used_property(self):
        subjects = self.__rows_for_most_used_property()['s'].values
        rows = self.__df.loc[self.__df['s'].isin(subjects)]
        return rows.loc[rows['p'] != self.__most_used_property]
        
    # the complement of rows of subjects that use the most used property of the loaded dataframe of a node
    def __complement_for_subjects_using_most_used_property(self):
        subjects = self.__rows_for_most_used_property()['s'].values
        rows = self.__df.loc[~self.__df['s'].isin(subjects)]
        return rows
    
    # constructor taking a KG in pandas dataframe format
    def __init__(self, dataframe):
        Node.__counter += 1
        self.__identifier = Node.__counter
        self.__df = dataframe.copy()
        self.__most_used_property = self.__most_used_properties(1)[0]
        
        right_rows = self.__complement_for_subjects_using_most_used_property()
        if right_rows.empty:
            self.right = None
        else:
            self.right = Node(right_rows)
        
        left_rows = self.__rows_for_subjects_using_most_used_property()        
        if left_rows.empty:
            self.left = None
        else:
            self.left = Node(left_rows)

    # print some interesting stats of the subgraph this node represents           
    def print_stats(self):
        print("======================================================")
        print("node: ", self.__identifier)
        print("number of rows: ", len(self.__df))
        print("distinct property count: ", len(self.__properties_unique()))
        print("distinct subject count: ", len(self.__subjects_unique()))
        print("most used property:", self.__most_used_property)
        print("======================================================")

    # print a graphviz representation of the constructed tree    
    def print_graph(self, dot = Digraph()):
        dot.node(name=str(self.__identifier), label=str(self.__most_used_property), URL=self.__most_used_property)
        if self.left != None:
            self.left.print_graph(dot)
            dot.edge(str(self.__identifier), str(self.left.__identifier), label=str(len(self.left.__subjects_unique())))
        if self.right != None:
            self.right.print_graph(dot)
            dot.edge(str(self.__identifier), str(self.right.__identifier), label=str(len(self.right.__subjects_unique())))
        return dot

    # loads a mapping in form of csv for the identifiers of the KG and stores them in internal dict mappings
    def load_map(self, mapping_file):
        self.subjects_mapping = {}
        self.properties_mapping = {}
        self.objects_mapping = {}
        with open(mapping_file) as f:
            with open(mapping_file, 'r') as read_obj:
                csv_reader = reader(read_obj, delimiter='\t')
                i = 0
                for row in csv_reader:
                    i+=1
                    if i % 1000000 == 0:
                        print(i, "mapping rows processed...")
                    identifier = int(row[0])
                    if identifier in self.__df['s'].values:
                        self.subjects_mapping[identifier] = row[1]
                        print('found in subjects: ', identifier, ":", row[1])
                    if identifier in self.__df['o'].values:
                        self.objects_mapping[identifier] = row[1]
                        print('found in objects: ', identifier, ":", row[1])
                    if identifier in self.__df['p'].values:
                        self.properties_mapping[identifier] = row[1]
                        print('found in properties: ', identifier, ":", row[1])

    # applies internal mappings to the rows of the internal dataframe - recursively         
    def apply_map(self):
        if self.subjects_mapping and self.properties_mapping and self.objects_mapping:   
            self.map_replace(self.subjects_mapping, self.objects_mapping, self.properties_mapping)
        else:
            raise Exception("Apply map require a loaded map! Use load_map() first!")
        
    def map_replace(self, subjects, objects, properties):
        self.__df.replace({ "s": subjects }, inplace=True)
        self.__df.replace({ "o": objects }, inplace=True)
        self.__df.replace({ "p": properties }, inplace=True)
        self.__most_used_property = self.__most_used_properties(1)[0]
        if self.left:
            self.left.map_replace(subjects, objects, properties)
        if self.right:
            self.right.map_replace(subjects, objects, properties)
    
    # stores this object for later use as a binary representation
    def serialize(self, filename):
        with open(filename, "wb") as file:
            pickle.dump(self, file)
            
    # loads object for reuse from a binary representation - returns a full object of type Node()
    def deserialize(filename):
        with open(filename, "rb") as file:
            return pickle.load(file)
df = pd.read_csv(raw)
tree = Node(df)

In [35]:
#tree.load_map(mapping)

In [36]:
with open('property_mapping.bin', "rb") as file:
    properties_mapping = pickle.load(file)
    tree.map_replace({}, {}, properties_mapping)
#with open('property_mapping.bin', "wb") as file:
#    pickle.dump(tree.properties_mapping, file)

In [37]:
tree.print_stats()

node:  1
number of rows:  7967
distinct property count:  844
distinct subject count:  3957
most used property: http://schema.org/about


In [38]:
dot = tree.print_graph(Digraph())

In [39]:
dot.body

['\t1 [label="http://schema.org/about" URL="http://schema.org/about"]\n',
 '\t2 [label="http://www.wikidata.org/prop/direct/P106" URL="http://www.wikidata.org/prop/direct/P106"]\n',
 '\t2799 [label="http://www.wikidata.org/prop/P106" URL="http://www.wikidata.org/prop/P106"]\n',
 '\t3056 [label="http://www.wikidata.org/prop/P27" URL="http://www.wikidata.org/prop/P27"]\n',
 '\t3307 [label="http://www.wikidata.org/prop/direct/P27" URL="http://www.wikidata.org/prop/direct/P27"]\n',
 '\t3357 [label="http://www.wikidata.org/prop/P19" URL="http://www.wikidata.org/prop/P19"]\n',
 '\t3384 [label="http://www.wikidata.org/prop/direct/P1411" URL="http://www.wikidata.org/prop/direct/P1411"]\n',
 '\t3591 [label="http://www.wikidata.org/prop/P1411" URL="http://www.wikidata.org/prop/P1411"]\n',
 '\t3622 [label="http://www.wikidata.org/prop/direct/P69" URL="http://www.wikidata.org/prop/direct/P69"]\n',
 '\t3623 [label="http://www.wikidata.org/prop/direct/P26" URL="http://www.wikidata.org/prop/direct/P2

In [40]:
dot